In [ ]:
import os
import sys

In [2]:
import ase.db
import numpy as np
import torch
from pymatgen.io.ase import AseAtomsAdaptor

In [3]:
test_db = ase.db.connect("../data/test_structs_1k.db")

In [4]:
test_sys = test_db.get_atoms(5)

In [6]:
def time_get_neighbor_list(atoms_db, radius=6):
    for row in atoms_db.select():
        atoms = row.toatoms()
        structure = AseAtomsAdaptor.get_structure(atoms)
        
        neighbors = structure.get_neighbor_list(r=radius)

In [7]:
def time_get_all_neighbors(atoms_db, radius=6):
    for row in atoms_db.select():
        atoms = row.toatoms()
        structure = AseAtomsAdaptor.get_structure(atoms)
        
        neighbors = structure.get_all_neighbors(r=radius, include_index=True)

In [8]:
test_structure = AseAtomsAdaptor.get_structure(test_sys)

In [78]:
%timeit nb1 = test_structure.get_neighbor_list(r=6.0)

513 µs ± 4.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [79]:
%timeit nb2 = test_structure.get_all_neighbors(r=6.0, include_index=True)

1.33 ms ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [94]:
%timeit test_sys.get_all_distances(mic=True)

980 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [88]:
%timeit nb1_fun = time_get_neighbor_list(test_db)

1.52 s ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
%timeit nb2_fun = time_get_all_neighbors(test_db)

5.5 s ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# appears the get_neighbor_list method is at least 3x faster

ASE Neighborlist

In [99]:
from ase.neighborlist import NeighborList, NewPrimitiveNeighborList, PrimitiveNeighborList

In [110]:
def ase_neighborlist(image):
    n = NeighborList(cutoffs=[6.0 / 2.0] * len(test_sys), 
                     skin=0, self_interaction=False, 
                     primitive=NewPrimitiveNeighborList, 
                     bothways=True)
    n.update(image)
    neighbors = [n.get_neighbors(index) for index in range(len(test_sys))]
    return neighbors

def ase_distances(image):
    return image.get_all_distances(mic=True)

def all_ase_neighborlist(atoms_db):
    for row in atoms_db.select():
        atoms = row.toatoms()
        n = NeighborList(cutoffs=[6.0 / 2.0] * len(atoms), 
                         skin=0, self_interaction=False, 
                         primitive=NewPrimitiveNeighborList, 
                         bothways=True)
        n.update(atoms)
        neighbors = [n.get_neighbors(index) for index in range(len(atoms))]

In [105]:
%timeit ase_neighbors = ase_neighborlist(test_sys)

4.04 ms ± 37.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [106]:
%timeit adist = ase_distances(test_sys)

994 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [111]:
%timeit all_ase_neighborlist(test_db)

10.8 s ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
